In [ ]:
import numpy as np
import zipfile
import os
import shutil

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# Extract the provided zip file
with zipfile.ZipFile('archive (3).zip', 'r') as zip_ref:
    zip_ref.extractall('archive (3)')

# Define directories for train, validation, and test sets
train_dir = 'archive (3)/train'
val_dir = 'archive (3)/validation'
test_dir = 'archive (3)/test'

# Create validation directory if it doesn't exist
if not os.path.exists(val_dir):
    os.makedirs(val_dir)

# Define the ratio of data to move to the validation set
validation_split = 0.2

# Move a portion of the training data to the validation directory
for class_name in os.listdir(train_dir):
    class_dir = os.path.join(train_dir, class_name)
    files = os.listdir(class_dir)
    num_validation = int(len(files) * validation_split)
    validation_files = files[:num_validation]
    for file in validation_files:
        src = os.path.join(class_dir, file)
        dst = os.path.join(val_dir, class_name)
        os.makedirs(dst, exist_ok=True)
        shutil.move(src, dst)

# Image dimensions
img_height, img_width = 224, 224
batch_size = 32

# Preprocess and augment data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load pre-trained VGG16 model without top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
model = Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          epochs=10,
          validation_data=val_generator,
          validation_steps=val_generator.samples // batch_size)

# Evaluate the model
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)


Found 1021 images belonging to 2 classes.
Found 254 images belonging to 2 classes.
Epoch 1/10
31/31 [==============================] - 694s 22s/step - loss: 1.5003 - accuracy: 0.6906 - val_loss: 0.2754 - val_accuracy: 0.9018
Epoch 2/10
31/31 [==============================] - 687s 22s/step - loss: 0.4246 - accuracy: 0.8342 - val_loss: 0.8364 - val_accuracy: 0.6339
Epoch 3/10
31/31 [==============================] - 662s 21s/step - loss: 0.4653 - accuracy: 0.8069 - val_loss: 0.2634 - val_accuracy: 0.8929
Epoch 4/10
31/31 [==============================] - 661s 21s/step - loss: 0.3250 - accuracy: 0.8665 - val_loss: 0.2445 - val_accuracy: 0.8839
Epoch 5/10
31/31 [==============================] - 679s 22s/step - loss: 0.2290 - accuracy: 0.9052 - val_loss: 0.3461 - val_accuracy: 0.8661
Epoch 6/10
31/31 [==============================] - 677s 22s/step - loss: 0.2282 - accuracy: 0.9009 - val_loss: 0.2783 - val_accuracy: 0.8795
Epoch 7/10
31/31 [==============================] - 679s 22s/step